In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [2]:
%run -i data_generation/simple_linear_regression.py
%run -i model_configuration/v0.py
%run -i data_preparation/v0.py


In [2]:
true_b = 1
true_w = 2
N = 100
np.random.seed(42)
x = np.random.rand(N, 1)
print(x.shape)
epsilon = (-0.1 * np.random.randn(N, 1))
y = true_w * x + true_b + epsilon

(100, 1)


In [3]:
idx = np.arange(N)
np.random.shuffle(idx)
train_idx = idx[:int(0.8 * N)]
val_idx = idx[int(0.8 * N):]
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx],y[val_idx]

In [4]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [5]:
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)
print(type(x_train),type(x_train_tensor), x_train_tensor.type())

<class 'numpy.ndarray'> <class 'torch.Tensor'> torch.FloatTensor


In [1]:
# %model_training/v0.py

In [6]:
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1, 1)).to(device)
model.state_dict()

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])

In [7]:

lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction='mean')
epochs = 1000

In [8]:
epochs = 1000
for _ in range(epochs):
    model.train()
    predictions = model.forward(x_train_tensor)
    loss = loss_fn(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
print(model.state_dict())

OrderedDict([('0.weight', tensor([[2.0310]])), ('0.bias', tensor([0.9765]))])


In [26]:

def make_train_step_fn(model, loss_fn, optimizer):
    def perform_train_step_fn(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return perform_train_step_fn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = nn.Linear(1, 1).to(device)
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction='mean')
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)
train_step_fn

<function __main__.make_train_step_fn.<locals>.perform_train_step_fn(x, y)>

#### Custom Dataset with Tensor

In [27]:

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return(self.x[index], self.y[index])
    def __len__(self):
        return len(self.x)

In [16]:
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)
train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.6105]))


### Same result with TensorDataset

In [18]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
# print(train_data[0])

In [17]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)


In [28]:

n_epochs = 1000
losses = []
for _ in range(n_epochs):
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        mini_batch_loss = train_step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
loss = np.mean(mini_batch_losses)
losses.append(loss)


In [29]:
def mini_batch(device, data_loader,  step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    loss = np.mean(mini_batch_losses)
    return loss

In [30]:
n_epochs = 200
losses = []
for _ in range(epochs):
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)
print(model.state_dict())

OrderedDict([('weight', tensor([[2.0316]])), ('bias', tensor([0.9768]))])


In [31]:
torch.manual_seed(13)
x_train = torch.as_tensor(x).float()
y_train = torch.as_tensor(y).float()
dataset = TensorDataset(x_train, y_train)
ratio = .8

n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=16)


In [32]:
def make_val_step_fn(model, loss_fn):
    def perform_val_step_fn(x, y):
        model.val()
        yhat = model(x)
        loss = loss_fn(yhat)
        return loss.item()
    return perform_val_step_fn

In [33]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1, 1)).to(device)
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction='mean')
epochs = 1000
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)
make_val_fn = make_val_step_fn(model, loss_fn)